In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [24]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# Simulate measurement of a Bell inequality.

# We will be doing measurements on the state 1/sqrt(2)( |01> - |10> ).

# We measure the first qubit (qubit 1 in qiskit notation) with respect to the observables A0 and A1.
# A0 = Z and this corresponds to a standard basis measurement.
# A1 = X and this corresponds to a diagonal basis measurement.
# To measure in the diagonal basis, we apply H and then measure.
# BUT we are working with eigenvalues as measurement results, so when we get the result, 
# we convert 0 to +1 and we convert 1 to -1.

# We measure the second qubit (qubit 0 in qiskit notation) with respect to the observables B0 and B1.
# This is a bit more complicated. We have the eigenvectors from Lecture 8 Slide 14.
# What we have to do is work out a matrix that maps these eigenvectors to |0> and |1>, 
# then do a standard basis measurement.
# The matrix that maps |0> and |1> to the eigenvectors is easily found, because the columns are just the eigenvectors.
# Then the matrix want is the inverse of that one. 

# We can construct these matrices explicitly and then convert them into unitary operators for use in circuits.
# These are B0_transform and B1_transform below.

# We want to record the average values of various measurements, then combine them according to the formula on 
# Lecture 8 Slide 15.
# We can find each average by simulating the circuit with its measurement many times, adding up all the results and 
# dividing by the number of trials.

# Finally we get an experimental value for the formula on Slide 15. Theoretically this should be 2*sqrt(2), so 
# we are hoping to see something close to that (approximately 2.828) but definitely bigger than 2 which is the maximum 
# for a local hidden variable theory (Slide 8).

# Remember though that we are not running on quantum hardware - this is all just a classical simulation.
# But we can interpret the result as checking that qBraid is correctly simulating quantum mechanics.

# So, let's get started!

# Useful constants
root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


B0_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]
B0_transform = Operator(B0_transform_matrix) 


B1_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]
B1_transform = Operator(B1_transform_matrix) 

# Given a circuit c which prepares the state 1/sqrt(2)( |01> - |10> ) and measures it, 
# simulate it n times and calculate the average measurement result (using the conversion to +1 and -1)

def average(c,n): 
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run(compiled, shots=n)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    #print(counts)
    count00 = counts.get("00",0) 
    count01 = counts.get("01",0) 
    count10 = counts.get("10",0) 
    count11 = counts.get("11",0) 
    # The return value includes the conversion from measurement results 0,1 to +1,-1
    # Each 00 means a value of  1 (+1 * +1)
    # Each 01 means a value of -1 (+1 * -1)
    # Each 10 means a value of -1 (-1 * +1)
    # Each 11 means a value of  1 (+1 * +1)
    return (count00 - count01 - count10 + count11) / n 

# Construct a circuit that prepares the state  1/sqrt(2)( |01> - |10> )
def entangledPair():
    q = QuantumCircuit(2) 
    q.h(0)
    q.cx(0,1)
    return q

# The number of simulations to use for averaging
N = 100

# Construct the circuit for measuring A0 B0, simulate it and calculate the average result
circuit_A0_B0 = entangledPair() 
# Don't transform qubit 0
# Transform qubit 1 by B0_transform
circuit_A0_B0.unitary(B0_transform_matrix,[1])
# Measure both qubits
circuit_A0_B0.measure_all()
A0_B0_average = average(circuit_A0_B0,N) 
#print(A0_B0_average)

# Construct the circuit for measuring A0 B1, simulate it and calculate the average result
circuit_A0_B1 = entangledPair() 
# Don't transform qubit 0
# Transform qubit 1 by B1_transform
circuit_A0_B1.unitary(B1_transform_matrix,[1])
# Measure both qubits
circuit_A0_B1.measure_all()
A0_B1_average = average(circuit_A0_B1,N) 
#print(A0_B1_average)

# Construct the circuit for measuring A1 B0, simulate it and calculate the average result
circuit_A1_B0 = entangledPair() 
# Transform qubit 0 by H
circuit_A1_B0.h(0)
# Transform qubit 1 by B0_transform
circuit_A1_B0.unitary(B0_transform_matrix,[1])
# Measure both qubits
circuit_A1_B0.measure_all()
A1_B0_average = average(circuit_A1_B0,N) 
#print(A1_B0_average)

# Construct the circuit for measuring A1 B1, simulate it and calculate the average result
circuit_A1_B1 = entangledPair() 
# Transform qubit 0 by H
circuit_A1_B1.h(0)
# Transform qubit 1 by B0_transform
circuit_A1_B1.unitary(B1_transform_matrix,[1])
# Measure both qubits
circuit_A1_B1.measure_all()
A1_B1_average = average(circuit_A1_B1,N)
#print(A1_B1_average)

# The final calculuation
result = abs(A0_B0_average + A0_B1_average + A1_B0_average - A1_B1_average)

print(result)



2.838
